# Линейная регрессия

Здесь хороший повод поговорить о математике, но меня просили не. Что ж, надеюсь, это не помешает вам сопоставить теорию с практикой. 

Суть: у нас есть несколько параметров и есть некоторое числовое значение, зависимость которого от этих самых параметров мы и хотим узнать. Например: есть количество квартир в доме, есть площадь парковки возле дома и есть площадь конкретной квартиры. По этим трём параметрам нужно найти стоимость этой квартиры. Пара из трёх параметров и цены квартиры составляют одну запись/один индивид. Таких индивидов у нас несколько (желательно довольно много). 

## Цели линейной регрессии

Собственно линейную регрессию используют для двух целей:
* Предсказать значение нового индивида при известных параметрах
* Объяснить как влияет каждый из параметров на искомое значение

## Когда её используют

Обычно её используют для получения первых выводов из данных, перед тем как начать использовать ~~нормальные~~ более сложные модели. 

## Какой у нас план?

В виду ограничений, будем пытаться разобраться по ходу, надеюсь, вам будет понятно. Вообще будет здорово, если освежите теоретические основы, но постараюсь всё показать на живом примере.

## Библиотека 

По традиции, нам нужно для начала установить библиотеку. В нашем случае на выбор предоставляется две библиотеки: `statsmodels` и `scikit-learn` (читают *склёрн*). Первая хороша для тех, кто дружит с матстатом, поэтому это не наш случай и выбираем вторую. Функционал этих библиотек неограничен исключительно линейной регрессией, но пока мы рассмотрим только её. 

In [ ]:
# !pip install scikit-learn

### Импорт

Внезапно, импортируем мы её не как `import scikit-learn` и даже не `import sklearn`. Библиотека довольно объёмна, а нам нужны только отдельные её модули, даже отдельные части её модулей. Для большей целостности импортируем всё необходимое здесь, а по ходу дела буду объяснять, что именно мы здесь набрали.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Начало работы / обработка датасета

Самое первое, что необходимо сделать $\text{---}$ так это узнать какие данные у нас есть и что мы хотим вообще найти. 

> Online property companies offer valuations of houses using machine learning techniques. The aim of this report is to predict the house sales in King County, Washington State, USA using Multiple Linear Regression (MLR). The dataset consisted of historic data of houses sold between May 2014 to May 2015.

В общем, по данным о недвижке будем предсказывать стоимость.

In [2]:
file_name = 'kc_house_data.csv'

df = pd.read_csv(file_name)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21611 non-null  float64
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long  

Сразу удалим то, что нам не нужно. 
* `id` $\text{---}$ идентификационный код никак не может повлиять на стоимость квартиры.
* `zipcode` $\text{---}$ это америакнский почтовый индекс, его можно использовать в анализе, но *feature exploration* такого уровня выходит за рамки нашего курса.
* `lat` и `long` $\text{---}$ широта и долгота, та же ситуация, что и выше. 

In [3]:
df.drop(['id', 'zipcode', 'lat', 'long'], axis=1, inplace=True)
df['price'] = df.pop('price') # для красоты поставим в конец
df.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,price
0,20141013T000000,3,1.00,1180,5650,1.0,0,0,3,7,1180.0,0,1955,0,1340,5650,221900.0
1,20141209T000000,3,2.25,2570,7242,2.0,0,0,3,7,2170.0,400,1951,1991,1690,7639,538000.0
2,20150225T000000,2,1.00,770,10000,1.0,0,0,3,6,770.0,0,1933,0,2720,8062,180000.0
3,20141209T000000,4,3.00,1960,5000,1.0,0,0,5,7,1050.0,910,1965,0,1360,5000,604000.0
4,20150218T000000,3,2.00,1680,8080,1.0,0,0,3,8,1680.0,0,1987,0,1800,7503,510000.0


Теперь разберёмся с оставшимися:
* `bedrooms` и `bathrooms` $\text{---}$ соответственно количество спален и ванных комнат. Внезапно количество ванных комнат не обязательно натуральное число, но что с них взять $\text{---}$ у них шоколадные батончики в дюймах и рутбир в пинтах. 
* `sqft_living` и `sqft_lot` $\text{---}$ соответственно площадь жилых помещений и земельных участков. 
* `sqft_living15` и `sqft_lot15` $\text{---}$ соответственно площадь жилых помещений и земельных участков, но в этот раз не конкретной недвижки, а ближайших 15 объектов. 
* `sqft_basement` и `sqft_above` $\text{---}$ соответственно площадь подвальных помещений и надземных. 
* `floors` $\text{---}$ количество этажей, та же история, что и с ванными комнатами.
* `waterfront` $\text{---}$ находится возле набережной.
* `viewfront`, `condition` и `grade` $\text{---}$ те или иные качественные оценки дома и окружающего вида. 
* `date` $\text{---}$ дата продажи дома.

Данные у нас с мая 14-ого по май 15-ого, допустим, что различия между месяцами больше, чем различия внутри месяца, поэтому преобразуем дату в строку вида `MM.YY`, сделаем из этого категориальную переменную. Вообще правильно было бы сделать несколько предположений, попробовать объединить по сезонам, кварталам, полугодиям, но в силу задач нашего курса оставим эту важную работу вам. Можете потом сообщить результаты.  

In [ ]:
df['date'] = df['date'].str[2:4] + '.' + df['date'].str[4:6]
df.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15,price
0,14.10,3,1.00,1180,5650,1.0,0,0,3,7,1180.0,0,1955,0,1340,5650,221900.0
1,14.12,3,2.25,2570,7242,2.0,0,0,3,7,2170.0,400,1951,1991,1690,7639,538000.0
2,15.02,2,1.00,770,10000,1.0,0,0,3,6,770.0,0,1933,0,2720,8062,180000.0
3,14.12,4,3.00,1960,5000,1.0,0,0,5,7,1050.0,910,1965,0,1360,5000,604000.0
4,15.02,3,2.00,1680,8080,1.0,0,0,3,8,1680.0,0,1987,0,1800,7503,510000.0


Сохраним преобразованные данные, чтобы не проводить вышеописанное каждый раз. В нашем случае не критично, но привычка полезная.

In [ ]:
save_file_name = 'kc_house_data_mod.csv'
df.to_csv(save_file_name, index=False)
df = pd.read_csv(save_file_name)

Далее разделим датасет на обучающий и проверочный набор данных. В проверочный набор данных мы не лезем. Совсем не лезем. Даже не смотрим. Это нечестно. Так делать не надо. 

In [52]:
seed = 1
train, test = train_test_split(df, train_size=0.8, random_state=seed)

Вообще, с точки зрения математики, необходимо обратить внимание на следующие признаки: `sqft_living`, `sqft_basement` и `sqft_above`. Дело в том, что они отчасти перекрывают друг друга. Если подвальные помещения оборудованы под жилые, то площадь подвального помещения уже будет учтена в площади жилых помещений. Если подвала нет, то площадь надземных помещений (как мы увидим далее) часто равна площади жилых помещений, тем самым она будет учтена дважды. В некотором смысле мы получаем такое страшное словосочетание как "линейная комбинация", а применяя метод линейной регрессии желательно не иметь в своих данных линейных комбинаций, т.к. есть методы, которые вообще не сойдутся при таких условиях. Мы попытаемся как-то разрешить эту проблему, но предупреждаю сразу, что я не буду пытаться достичь лучшего результата, поскольку, ну, это просто учебный пример. Посмотрим что мы можем вытащить.

In [53]:
# Количество объектов, где сумма площади надземных и подземных помещений не
# равна площади надземных помещений   
train[(train['sqft_above'] + train['sqft_basement']) 
      != train['sqft_living']].shape[0]

2

In [40]:
# Количество объектов, где площадь надземных помещений не равна площади жилых 
# помещений, при отсутствующем подвале (очевидно, что это один из тех двух
# случаев).
len(train[(train['sqft_above'] != train['sqft_living']) 
          & (train['sqft_basement'] == 0)])

1

И если вы не сказали сейчас: "Так, минуточку", то это было зря. Можете сказать это сейчас, а я объясню почему. Вернёмся к выводу информации о датасете. Я дублировать не буду, сами посмотрите. Если присмотреться внимательно, то можно заметить, что у нас 

In [43]:
type(df['date'][0])

str